In [23]:
import pandas as pd
import numpy as np
from my_api import *
import matplotlib.pyplot as plt

In [24]:
id = pd.MultiIndex.from_product([['fro', 2], [100, 300, 500]])
df = pd.DataFrame(index = id)

In [30]:
T = 300
rela = bool(0)
cov_str = 'Cai2011Adaptive_Model2_my'
# cov_str = 'Cai2011Adaptive_Model1'

In [27]:
simu_str = 'lx_band'
data_path = 'data_2023-4-1/'
for N in [100, 300, 500]:
    S = gen_S_Cai2011Adaptive_Model1(N)
    
    for ord in ['fro', 2]:
        normS = np.linalg.norm(S, ord)
        
        for eta in [0.5, 0.8, 1]:
            cov_params = [cov_str]
            simu_params = [simu_str, eta]
            file_name = f"{ord},S,{N},{T},{cov_params},{simu_params}.txt"
            file_name = data_path + file_name
            data = np.loadtxt(file_name)

            col = str(simu_params)
            if col not in df.columns:
                df[col] = None
            if not rela:
                data *= normS # absolute error
            df[col][ord, N] = f"{data.mean() :.2f}({data.std() :.2f})"

In [28]:
simu_str = 'LSRthreshold'
data_path = 'data_LSRthreshold_2023-4-6/'
for N in [100, 300, 500]:
    S = gen_S_Cai2011Adaptive_Model1(N)
    
    for ord in ['fro', 2]:
        normS = np.linalg.norm(S, ord)
        
        for tau in [0.2]:
            # for prob in [0.99, 0.9, 1]:
            #     for qrob in [0, 0.01, 0.1]:
            for prob in [1, 0.99]:
                for qrob in [0, 0.01]:
                    cov_params = [cov_str]
                    simu_params = [simu_str, tau, prob, qrob, 'brute']
                    file_name = f"{ord},S,{N},{T},{cov_params},{simu_params}.txt"
                    file_name = data_path + file_name
                    
                    try:
                        data = np.loadtxt(file_name)
                    except FileNotFoundError:
                        continue

                    col = str(simu_params)
                    if col not in df.columns:
                        df[col] = None
                    if not rela:
                        data *= normS # absolute error
                    df[col][ord, N] = f"{data.mean() :.2f}({data.std() :.2f})"

In [17]:
simu_str = '' # to be determined
data_path = 'data_OtherMethods_4-5/'

for N in [100, 300, 500]:
    S = gen_S_Cai2011Adaptive_Model1(N)
    
    for ord in ['fro', 2]:
        normS = np.linalg.norm(S, ord)

        # from Cai2011_other import Other_Methods
        for simu_str in ['Sample', 'Soft Threshold', 'Hard Threshold', 'Linear Shrink', 'Nonlinear Shrink']:
            cov_params = [cov_str]
            simu_params = [simu_str]
            file_name = f"{ord},S,{N},{T},{cov_params},{simu_params}.txt"
            file_name = data_path + file_name
            try:
                data = np.loadtxt(file_name)
            except FileNotFoundError:
                continue

            col = str(*simu_params)
            if col not in df.columns:
                df[col] = None
            
            if not rela:
                data *= normS # absolute error
            df[col][ord, N] = f"{data.mean() :.2f}({data.std() :.2f})"

In [29]:
df

['lx_band', 0.5] ['lx_band', 0.8] ['lx_band', 1]  \
fro 100       0.21(0.01)       0.13(0.01)     0.12(0.01)   
    300       0.25(0.00)       0.18(0.00)     0.17(0.00)   
    500       0.28(0.00)       0.21(0.00)     0.20(0.00)   
2   100       0.41(0.02)       0.24(0.02)     0.21(0.02)   
    300       0.32(0.01)       0.25(0.01)     0.25(0.01)   
    500       0.31(0.01)       0.23(0.01)     0.23(0.01)   

        ['LSRthreshold', 0.2, 1, 0, 'brute']  \
fro 100                           0.35(0.02)   
    300                           0.61(0.03)   
    500                           0.79(0.03)   
2   100                           0.51(0.04)   
    300                           0.73(0.05)   
    500                           0.87(0.05)   

        ['LSRthreshold', 0.2, 0.99, 0.01, 'brute']  
fro 100                                 0.35(0.02)  
    300                                 0.61(0.03)  
    500                                 0.79(0.04)  
2   100                                 0.52(0.04)  
    300                                 0.72(0.05)  
    500                                 0.87(0.05)

In [19]:
new_cols = []
for col in df.columns:
    try:
        col = eval(col)
    except Exception:
        pass

    if type(col) == str:
        new_col = col
    elif 'lx_band' in col:
        eta = col[1]
        new_col = fr'Augmented Band ($\eta={eta}$)'
    elif 'LSRthreshold' in col:
        t, p, q = col[1:-1]
        new_col = fr'Augmented Threshold ($\tau={t}, p={p}, q={q}$)' # The r in fr, means raw string. Otherwise I should write \\tau instead of \tau.
    new_cols.append(new_col)
df.columns = new_cols

In [20]:
buf = 'latex/' + cov_str + '.tex'
caption = cov_str

with pd.option_context("max_colwidth", 1000, 'display.width', 50):
    df.to_latex(buf = buf, 
                index = True, na_rep = ' ', sparsify = True, 
             column_format = 'll|' + 'p{2cm}' * df.shape[1], 
             longtable = False, 
             escape = False, encoding = 'utf-8', 
             caption = caption, label = "my label", 
             position = 'htbp')
# escape参数，防止字符被转换使得我设的数学模式被破坏

C:\Users\lx\AppData\Local\Temp\ipykernel_25232\4238430270.py:5: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  df.to_latex(buf = buf,


In [21]:
pd.get_option('display.width')

80

In [22]:
lines = []

with open(buf, 'r') as fp:
    lines = fp.readlines()

s1 = r'\resizebox{\linewidth}{!}{' + '\n'
s2 = r'}' + '\n'
lines.insert(4, s1)
lines.insert(-1, s2)

with open(buf[:-4] + '_processed' + '.tex', 'w') as fp:
    fp.writelines(lines)
        # if not write_flag:
        #     if r'begin{tabular}' in line:
